BLE生データから個人の移動経路を粗く抽出．
ポテンシャル場を求めるコードと途中まで同じ

In [1]:
import pandas as pd 
import numpy as np 
import os 

In [2]:
basepath = '/Users/takahiromatsunaga/res2023/bledata/ble_timegroup' #/20230130_08' # /20230130_0845.csv'
day_stamp = '20230130'
time_stamp = '17'
date_stamp = f'{day_stamp}_{time_stamp}'
df = pd.read_csv(os.path.join(basepath, date_stamp, f'{date_stamp}30.csv'))
df_ble = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_nw_detail/ble_nw.csv')
df_node = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/ver0707/micro_node_post.csv')
df_link = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/ver0707/micro_link_post.csv')
df_matrix = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/ver0707/nodebased_matrix_post.csv')

In [3]:
bleid_list = sorted(df['ID'].unique())
nodeid_list = sorted(df_node['nodeid'].unique())

各セルにおける主流の方向を判別


In [4]:
# 個人ごとに分割
grouped2 = df.groupby('MAC')
df_list2 = [group.reset_index(drop = True) for name, group in grouped2]
print(len(df_list2))

89963


In [5]:
# 検出ビーコン数が2以下の場合消去
filtered_df_list2 = []
for u in range(len(df_list2)):
    dfu = df_list2[u]
    if dfu['ID'].nunique() <= 2:
        continue
    dfu_copy = dfu.copy()
    filtered_df_list2.append(dfu_copy)

print(len(filtered_df_list2)) # 23070くらい（半減した）

39729


20秒間隔でtimestep与えて絞り込む

In [6]:
# 10 or 20秒ごとにサンプリング
# 検出時間が3step以下のものは削除
import datetime 
import pytz
initial_time = pd.Timestamp(f'2023-01-30 {time_stamp}:30:00')
initial_time = initial_time.tz_localize('Asia/Tokyo')  # JSTとしてローカライズ
seconds = 20

#### dfu_copyにtime_diff列とtimestep列を付与
filtered_df_list3 = []
for dfu in filtered_df_list2:
    dfu_copy = dfu.copy()
    dfu_copy['time'] = pd.to_datetime(dfu_copy['time']).dt.tz_convert('Asia/Tokyo') 
    ## timeの昇順で並べ替えたい
    # dfu_copy = sorted.dfu_copy(key = 'time')
    
    # すでに形上では+9:00がついているものの，df.tz_convert以降をつけないと正式には認識されない！！
    dfu_copy['time_diff'] = (dfu_copy['time'] - initial_time).dt.total_seconds()
    dfu_copy['timestep'] = (dfu_copy['time_diff'] // seconds).astype(int) # dt.tz_localize('Asia/Tokyo') 
    
    dfu_copy = dfu_copy.sort_values(by='time').reset_index(drop=True)

    if dfu_copy['timestep'].nunique() <= 2:
        continue

    filtered_df_list3.append(dfu_copy)

print(len(filtered_df_list3))

37594


各個人について20秒ごとにBLEビーコンNW上での観測重みグラフを作成
その差分から移動ベクトルを抽出したいが．．

In [7]:
res_dict = {} # 個人ごとに各timestepでBLEスコアを入れるT*Bのグラフ（行列）を格納
B = len(bleid_list) # Bはビーコン数
TT = int(1800 / seconds) # 全部のtimestep数
print(f'seconds={seconds}, TT={TT}')
for dfi in filtered_df_list3: # 個人ごと
    res_mat = np.zeros((TT, B)) # 各timestepでの各ビーコンのスコア
    # 時間ごとに分けて集計
    grouped3 = dfi.groupby('timestep')
    mac = dfi.loc[0, 'MAC']
    dfit_list = [group.reset_index(drop = True) for name, group in grouped3]
    for dfit in dfit_list: # 各timestep内で
        ts = dfit.loc[0, 'timestep']
        it_dict = {} # 各ビーコンの累積スコアを記録する辞書
        for i in range(len(dfit)):
            beacon_id = dfit.loc[i, 'ID']
            beacon_ix = bleid_list.index(beacon_id)
            rssi = dfit.loc[i, 'RSSI']
            res_mat[ts-1, beacon_ix] += (100+rssi)/10
    res_dict[mac] = res_mat 

seconds=20, TT=90


res_dictに入っている個々人のスコア行列から時刻ごとの重心を出す

In [8]:
# res_matから重心を出す関数を定義
B = len(bleid_list)
ble_loc_mat = np.zeros((B, 3)) # ベクトルで持ってたほうが楽なはず
for i in range(B):
    bid = bleid_list[i] # ビーコンのid
    x, y, z = df_ble[df_ble['ID'] == bid]['x'].iloc[0], df_ble[df_ble['ID'] == bid]['y'].iloc[0], df_ble[df_ble['ID'] == bid]['floor'].iloc[0]
    ble_loc_mat[i, 0] = x
    ble_loc_mat[i, 1] = y
    ble_loc_mat[i, 2] = z

ble_x_vec = ble_loc_mat[:, 0]
ble_y_vec = ble_loc_mat[:, 1]
ble_z_vec = ble_loc_mat[:, 2]

x_tiled = np.tile(ble_x_vec, (TT, 1))
y_tiled = np.tile(ble_y_vec, (TT, 1))
z_tiled = np.tile(ble_z_vec, (TT, 1))


def gravity_center(res_mat): # res_matの形状はTT*B
    w_sum_vec = np.sum(res_mat, axis=1) # 重みの和
    w_sum_vec = (w_sum_vec==0) + (w_sum_vec!=0) * w_sum_vec
    xw_mat = res_mat * x_tiled
    yw_mat = res_mat * y_tiled
    zw_mat = res_mat * z_tiled
    x_center_vec = np.sum(xw_mat, axis=1) / w_sum_vec
    y_center_vec = np.sum(yw_mat, axis=1) / w_sum_vec
    z_center_vec = np.sum(zw_mat, axis=1) / w_sum_vec
    return x_center_vec, y_center_vec, z_center_vec

def interpolate_missing_values(vec): # 欠損部分（0）の補間を行う関数 ### これで欠損のあるデータもそれっぽく使える！
    non_zero_indices = np.nonzero(vec)[0]
    zero_indices = np.where(vec == 0)[0]
    
    if len(non_zero_indices) == 0: # 非零成分がない場合は無視
        return vec  

    interpolated_vec = vec.copy()
    for zero_idx in zero_indices:
        # 前後の非零成分を見つける
        previous_non_zero_idx = non_zero_indices[non_zero_indices < zero_idx][-1] if len(non_zero_indices[non_zero_indices < zero_idx]) > 0 else None
        next_non_zero_idx = non_zero_indices[non_zero_indices > zero_idx][0] if len(non_zero_indices[non_zero_indices > zero_idx]) > 0 else None
        
        if previous_non_zero_idx is not None and next_non_zero_idx is not None:
            # 線形補間
            interpolated_value = vec[previous_non_zero_idx] + (vec[next_non_zero_idx] - vec[previous_non_zero_idx]) * (zero_idx - previous_non_zero_idx) / (next_non_zero_idx - previous_non_zero_idx)
            interpolated_vec[zero_idx] = interpolated_value

    return interpolated_vec

res_dict4 = {}

## 欠損観測間を補完
for mac, res_mat in list(res_dict.items()):
    x, y, z = gravity_center(res_mat) # ベクトル演算にしたのえらい
    x_corrected, y_corrected, z_corrected = interpolate_missing_values(x), interpolate_missing_values(y), interpolate_missing_values(z)
    res_dict4[mac] = [x_corrected, y_corrected, z_corrected]

#print(list(res_dict4.items())[0]) # 最初のuserのMACアドレス
# res_list = list(res_dict.items())[0][1]
# #print(res_list[0]) # xの座標
# print(list(res_dict.items())[0][1])

初期重心・最終重心に最寄りのノードから判定して一致するmacを抜き出す方が良い
＜本流とは外れるが00_seikeiの上位互換版＞
またこの時のodを記録しておく（ここでOD判定する）
→odlist_makeの役目はここで与えた方が楽である．odを確定させればいいので

In [15]:
homenodes = [i for i in range(1, 16)]
# chuohachinodeds = [28, 29, 39] # ちょっと絞りすぎかもしれないが
chuohachiminaminodes = [28, 29, 35, 36, 39]
saikyo_beacon = [19, 22, 23]
gatenodes = [df_node[df_node['floor'] != 20.5]]

In [10]:
gatenodes = df_node[df_node['floor'] != 20.5]['nodeid'].to_list()
print(gatenodes)


[24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 100]


In [11]:
from scipy.spatial import cKDTree

df_node_nonabs = df_node.copy()
df_node_nonabs = df_node_nonabs.iloc[:-1, :]
node_positions = df_node_nonabs[['x', 'y', 'floor']].values ### この書き方便利 # print(node_positions)
saikyo_nodelist = [i for i in range(1, 8)]
yamate_nodelist = [i for i in range(8, 16)]
# home_nodelist = [i for i in range(16, )]
df_node_saikyo = df_node_nonabs[df_node_nonabs['nodeid'].isin(saikyo_nodelist)]
df_node_yamate = df_node_nonabs[df_node_nonabs['nodeid'].isin(yamate_nodelist)]

node_positions_saikyo = df_node_saikyo[['x', 'y', 'floor']].values
node_positions_yamate = df_node_yamate[['x', 'y', 'floor']].values

kdtree = cKDTree(node_positions) # KDTreeを構築
kdtree_saikyo = cKDTree(node_positions_saikyo) # KDTreeを構築
kdtree_yamate = cKDTree(node_positions_yamate) # KDTreeを構築

### 途中の重心をどのノードに貼り付けるかのやつ
nonsaikyo_nodelist = [item for item in nodeid_list if item not in saikyo_nodelist]
nonyamate_nodelist = [item for item in nodeid_list if item not in yamate_nodelist]

df_node_nonsaikyo = df_node_nonabs[df_node_nonabs['nodeid'].isin(nonsaikyo_nodelist)] # ODが山手線の人用
df_node_nonyamate = df_node_nonabs[df_node_nonabs['nodeid'].isin(nonyamate_nodelist)] # ODが埼京線の人用

node_positions_nonsaikyo = df_node_nonsaikyo[['x', 'y', 'floor']].values
node_positions_nonyamate = df_node_nonyamate[['x', 'y', 'floor']].values

kdtree_nonsaikyo = cKDTree(node_positions_nonsaikyo) # KDTreeを構築
kdtree_nonyamate = cKDTree(node_positions_nonyamate) # KDTreeを構築

/Users/takahiromatsunaga/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


OD確定．

ただし山手線ホーム発で降車に限る．欠損も許容



In [16]:
# res4_dictにmacごとに重心座標ベクトルが入ってる
# res_dictにはビーコンスコアが入っている．res_dict, res_dict4, filtered_df_dict3は長さ同じ
# これまでに，各時刻のビーコンスコアと重心位置が算出されている
# odに応じて途中の重心のマッチングを変える

# 観測が連続かを判定する関数
def is_continuous(lst):
    lst = sorted(lst)
    for i in range(1, len(lst)):
        if lst[i] - lst[i-1] != 1:
            return False
    return True

## 結果貼りつけ用
final_observation_df = pd.DataFrame(columns=['MAC', 'ID', 'RSSI', 'time', 'time_diff', 'timestep', 'tt', 'o', 'd'])

direction = 'off' # 乗車の時 # direction = 'on' # 乗車の時

od_mac_dict = {}

for i in range(len(res_dict)): # 個人毎
    mac, (x_grav, y_grav, z_grav) = list(res_dict4.items())[i] ## 個人の重心の時系列データ
    dfi = filtered_df_list3[i] # 個人のデータ
    dfi_timestep_list = dfi['timestep'].unique()

    ################################################################
    ##### ここで非連続なuserは切ってる→potential場を出す際にはきらない ##### ポイント!!!
    ################################################################
    # if not is_continuous(dfi_timestep_list):
    #     continue

    #dfi['o'] = 0 # 初期値
    #dfi['d'] = 0
    mac2, res_mat = list(res_dict.items())[i] # 各時間のビーコン毎のスコア
    #print('res_mat', res_mat)
    #print('observation_omit', np.nonzero(res_mat))

    ## この前に非観測時刻のものをスキップする必要 # 埼京ビーコンを判別するだけなのでres_matのうち0の行は排除するのでいいや
    new_res_mat = res_mat[~np.all(res_mat == 0, axis=1)]
    #print('new_res_mat', new_res_mat)
    new_x_grav = np.nonzero(x_grav) #[~np.all(x_grav == 0)] #, axis=1)]
    new_y_grav = np.nonzero(y_grav) # [~np.all(y_grav == 0)] #, axis=1)]
    new_z_grav = np.nonzero(z_grav) # [~np.all(z_grav == 0)] #, axis=1)]
    #print(f'x_grav={new_x_grav}, y_grav={new_y_grav}, z_grav={new_z_grav}')

    init_loc = np.array((x_grav[new_x_grav[0][0]], y_grav[new_y_grav[0][0]], z_grav[new_z_grav[0][0]]))
    last_loc = np.array((x_grav[new_x_grav[0][-1]], y_grav[new_y_grav[0][-1]], z_grav[new_z_grav[0][-1]]))

    dist_init, idx_init = kdtree.query(init_loc) # dist_tt, idx_tt = kdtree.query(pos_tt)
    dist_last, idx_last = kdtree.query(last_loc)

    nearest_inint_nodeid = df_node_nonabs.iloc[idx_init]['nodeid'] # 時刻tの重心点と最寄りのノードidを返す
    nearest_inint_nodeix = nodeid_list.index(nearest_inint_nodeid)
    nearest_last_nodeid = df_node_nonabs.iloc[idx_last]['nodeid'] # 時刻tの重心点と最寄りのノードidを返す
    nearest_last_nodeix = nodeid_list.index(nearest_last_nodeid)

    ## 埼京を判別してホーム区別
    max_beacons = np.argmax(new_res_mat, axis = 1) # resmatの各列でスコア最大のもののindexを取得
    max_beacons_list = max_beacons.tolist() # indexのリスト
    max_beacons_id_list = [bleid_list[item] for item in max_beacons_list] # indexをidに変換
    
    indicator = 'yamate' # 初期値はyamateにしておく

    if any(item in saikyo_beacon for item in max_beacons_id_list):
        indicator = 'saikyo'
        continue # 申し訳ないけど埼京ホームは情報としてカット

    if direction == 'off':
        if nearest_inint_nodeid in homenodes and nearest_last_nodeid in chuohachiminaminodes:
            d_id = nearest_last_nodeid
            if indicator == 'saikyo':
                print('これには絶対にならないはず')
                dist_init_, idx_init_ = kdtree_saikyo.query(init_loc) # dist_tt, idx_tt = kdtree.query(pos_tt)
                nearest_inint_nodeid_ = saikyo_nodelist[idx_init_]
            elif indicator == 'yamate':
                dist_init_, idx_init_ = kdtree_yamate.query(init_loc) # dist_tt, idx_tt = kdtree.query(pos_tt)
                nearest_inint_nodeid_ = yamate_nodelist[idx_init_]
                #print(f'yamateある, init_idx={idx_init_}, init_nodeid={nearest_inint_nodeid_}') 
            
            o_id = nearest_inint_nodeid_ # homenodes_limitedから選ぶ
            #print(f'o_id={o_id}')
        else:
            continue

    elif direction == 'on':
        if nearest_inint_nodeid in chuohachiminaminodes and nearest_last_nodeid in homenodes:
            o_id = nearest_inint_nodeid
            if indicator == 'saikyo':
                # print('tree_responce', kdtree_saikyo.query(last_loc))
                dist_last_, idx_last_ = kdtree_saikyo.query(last_loc) # dist_tt, idx_tt = kdtree.query(pos_tt)
                nearest_last_nodeid_ = saikyo_nodelist[idx_last_]
            elif indicator == 'yamate':
                dist_last_, idx_last_ = kdtree_yamate.query(last_loc) # dist_tt, idx_tt = kdtree.query(pos_tt)
                nearest_last_nodeid_ = yamate_nodelist[idx_last_]
                # print('tree_responce', kdtree_saikyo.query(last_loc))
            # nearest_last_nodeid_ = df_node_nonabs.iloc[idx_last_]['nodeid'] # 時刻tの重心点と最寄りのノードidを返す
            d_id = nearest_last_nodeid_ # homenodes_limitedから選ぶ            else:
        else:
            continue
    
    #dfi['o'] = o_id
    #dfi['d'] = d_id
    dfi['tt'] = 0

    init_timestep = dfi.loc[0, 'timestep']
    for i in range(len(dfi)):
        current_timestep = dfi.loc[i, 'timestep']
        dfi.loc[i, 'tt'] = current_timestep - init_timestep + 1
    last_tt = dfi.loc[len(dfi)-1, 'tt']

    if last_tt >= 21: # ちょっと長いのでカット # 観測数は20以下→
        continue

    ### 辞書に結果格納 ###
    od_mac_dict[mac] = (o_id, d_id, last_tt)
    print('o_id, d_id, last_tt', o_id, d_id, last_tt)

    # columns_to_copy = ['MAC', 'ID', 'RSSI', 'time', 'time_diff', 'timestep', 'tt', 'o', 'd']
    # dfi_columfiltered = dfi[columns_to_copy].copy()

    # final_observation_df = pd.concat([dfi_columfiltered, final_observation_df], ignore_index=True)

####*****###------output------####****####
# final_observation_df.to_csv(os.path.join(basepath, date_stamp, f'{date_stamp}30_{seconds}sec_{direction}.csv' ))

o_id, d_id, last_tt 11 36.0 4
o_id, d_id, last_tt 12 36.0 9
o_id, d_id, last_tt 12 29.0 10
o_id, d_id, last_tt 8 39.0 9
o_id, d_id, last_tt 13 29.0 7
o_id, d_id, last_tt 13 29.0 9
o_id, d_id, last_tt 11 36.0 9
o_id, d_id, last_tt 13 29.0 6
o_id, d_id, last_tt 8 35.0 10
o_id, d_id, last_tt 12 28.0 7
o_id, d_id, last_tt 13 29.0 7
o_id, d_id, last_tt 11 36.0 10
o_id, d_id, last_tt 11 28.0 9
o_id, d_id, last_tt 13 28.0 12
o_id, d_id, last_tt 8 35.0 11
o_id, d_id, last_tt 14 28.0 16
o_id, d_id, last_tt 14 29.0 13
o_id, d_id, last_tt 13 29.0 7
o_id, d_id, last_tt 13 35.0 6
o_id, d_id, last_tt 8 36.0 14
o_id, d_id, last_tt 8 29.0 12
o_id, d_id, last_tt 11 29.0 16
o_id, d_id, last_tt 12 35.0 9
o_id, d_id, last_tt 12 28.0 8
o_id, d_id, last_tt 11 35.0 9
o_id, d_id, last_tt 13 35.0 7
o_id, d_id, last_tt 11 28.0 8
o_id, d_id, last_tt 11 36.0 13
o_id, d_id, last_tt 12 28.0 6
o_id, d_id, last_tt 8 35.0 16
o_id, d_id, last_tt 8 28.0 20
o_id, d_id, last_tt 8 36.0 10
o_id, d_id, last_tt 12 29.0 8
o_id

od_mat_dictと観測プリズム制約からres4_dictの各時刻における重心をNW上のノードに割り当てて各個人のノード系列を生成する

In [17]:
# 観測プリズムの前に普通のノード接続行列必要だった
N = len(df_node)
abs_id = 100
abs_ix = nodeid_list.index(abs_id)

In [23]:
x0, y0, z0 = list(res_dict4.items())[0][1]
print(x0, y0, z0)
# 非零成分のみ取り出す
non_zero_x0 = [value for value in x0 if value != 0]
non_zero_y0 = [value for value in y0 if value != 0]
non_zero_z0 = [value for value in z0 if value != 0]

print(non_zero_x0)
print(non_zero_y0)
print(non_zero_z0)


[     0.              0.              0.              0.
      0.              0.              0.              0.
      0.              0.              0.              0.
      0.              0.              0.              0.
      0.              0.              0.              0.
      0.              0.              0.              0.
 -11904.653      -11904.653      -11918.9825     -11923.507
 -11932.795025   -11942.08305    -11946.53921429 -11955.92737008
 -11957.90966135 -11955.07932143 -11958.11963576 -11964.26611111
 -11966.74945638 -11966.577           0.              0.
      0.              0.              0.              0.
      0.              0.              0.              0.
      0.              0.              0.              0.
      0.              0.              0.              0.
      0.              0.              0.              0.
      0.              0.              0.              0.
      0.              0.              0.              0.
      0.    

In [33]:
## od_mat_dictに入っているものを取り出していく．
# aのキーを順に見ていき、bの対応するアイテムを取得
res_dict5 = {}
count = 0
output_list = []

for mac in od_mac_dict.keys():
    if mac in res_dict4:
        (x_grav, y_grav, z_grav) = (res_dict4[mac]) ## これでいい？？
        # print(f"MACアドレス: {mac}, aのデータ: {a[mac]}, bのデータ: {b[mac]}")
    else: # なければカット
        continue
    
    ### 書き方これでいいのか？
    o_id, d_id, Ti = od_mac_dict[mac] # Tiまでにd_idに到着
    print('o_id, d_id, last_tt', o_id, d_id, Ti)
    
    # 観測プリズム作る
    o_ix = nodeid_list.index(o_id)
    d_ix = nodeid_list.index(d_id)
    
    I = np.eye(N) # 滞在可能
    for i in range(len(df_matrix)):
        kn = df_matrix.loc[i, 'k'] # node id
        an = df_matrix.loc[i, 'a']
        k = nodeid_list.index(kn) # node index
        a = nodeid_list.index(an)
        I[k, a] = 1  # 普通に滞在も可能→吸収と最後だけ省く

    I[:, abs_ix] = 0
    I[abs_ix, :] = 0
    I[abs_ix, abs_ix] = 1
    I[d_ix, abs_ix] = 1 # dからdへの滞在は認める（認めないで変なマッピングになっても困る）

    # 観測プリズム用意 ## odを与えたらプリズム制約．時刻Ti+1つまりTi回の遷移でaoからabsまで行く．もしくは時刻TiつまりTi-1回の遷移でaoからlastまで行く
    Idm = np.zeros((Ti, N, N)) # Ti回の遷移でo→dまで移動
    Itmlist = np.zeros((Ti+1, N, N))
    IIm = np.copy(I)
    Itmlist[0, :, :] = np.eye(N)

    for tt in range(1, Ti+1): 
        Itmlist[tt, :, :] = IIm 
        IIm = np.dot(IIm, I)
        IIm = np.where(IIm > 0, 1, 0)

    Ittmlist = np.zeros((Ti+1, N, N))
    for tt in range(Ti+1): 
        Ittmlist[tt, :, :] = np.transpose(Itmlist[(Ti+1) - tt - 1, :, :])

    for tt in range(Ti):
        if tt == 0:
            Idm[tt, o_ix, :] = I[o_ix, :] 
            continue
        alist = np.where(Itmlist[tt + 1, o_ix, :] == Ittmlist[tt + 1, d_ix, :])[0] # ここadじゃなくてaabsが適切では????
        for a in alist:
            if Itmlist[tt + 1, o_ix, a] == Ittmlist[tt + 1, d_ix, a]: # ここadじゃなくてaabsが適切では????
                klist = np.where(I[:, a] == 1)[0] 
                for k in klist:
                    if len(np.where(Idm[tt - 1, :, k] == 1)[0]) != 0:
                        Idm[tt, k, a] = 1 ### 時刻tにおいて，a(t)→a(t+1)への遷移がOKなところが1になっている     
    
    ## 時刻ごとに重心を見てidmの集合から最寄りのノードを確定させる→これをくりかえす
    ## 非零成分を取り出すか
    nonzero_xgrav = [value for value in x_grav if value != 0]
    nonzero_ygrav = [value for value in y_grav if value != 0]
    nonzero_zgrav = [value for value in z_grav if value != 0]
    
    # ラベル. 初期値はo, 毎回更新する
    current_node_id = o_id

    print('Ti=', Ti, 'len(nonzero_xgrav)', len(nonzero_xgrav))

    for t in range(1, (Ti)): # t=0の時はo_idで決定済みなのでパス # Ti == len(nonzero_xgrav)のはず
        t_loc = np.array([nonzero_xgrav[t], nonzero_ygrav[t], nonzero_zgrav[t]])
        # このタイミングで利用可能なノードたち
        current_node_ix = nodeid_list.index(current_node_id)
        available_nodes_ix = np.where(Idm[t, current_node_ix, :]==1)[0]
        available_nodes_id = [nodeid_list[item] for item in available_nodes_ix] # nodeidのリストになる
        print('available_nodes_id' , available_nodes_id)
        if len(available_nodes_id) == 0:
            print('次のユーザいこう')
            break
        
        # df_node_avail = df_node[df_node['nodeid'].isin(available_nodes_id)] # df_nodeのうち利用可能なノードのみ抽出
        df_node_avail = df_node[df_node['nodeid'].isin(available_nodes_id)].reset_index(drop=True)
        # print(df_node_avail)
        node_positions_avail = df_node_avail[['x', 'y', 'floor']].values
        kdtree_avail = cKDTree(node_positions_avail) # KDTreeを構築
        dist_avail, idx_avail = kdtree_avail.query(t_loc) 
        nearest_node_id = df_node_avail.loc[idx_avail, 'nodeid']
        current_node_id = nearest_node_id
        output_list.append([count, current_node_id])

    count += 1
    print('いった')

df_output = pd.DataFrame(output_list, columns=['id', 'node'])
df_output.to_csv(os.path.join(basepath, date_stamp, f'transit_{date_stamp}30_20sec_off.csv')) #, index=False))

o_id, d_id, last_tt 11 36.0 4
Ti= 4 len(nonzero_xgrav) 4
available_nodes_id []
次のユーザいこう
いった
o_id, d_id, last_tt 12 36.0 9
Ti= 9 len(nonzero_xgrav) 9
available_nodes_id [11, 12, 13]
available_nodes_id [10, 11, 12, 34]
available_nodes_id [9, 10, 11]
available_nodes_id [9, 31]
available_nodes_id [31, 32]
available_nodes_id [32]
available_nodes_id [33]
available_nodes_id [36]
いった
o_id, d_id, last_tt 12 29.0 10
Ti= 10 len(nonzero_xgrav) 10
available_nodes_id [11, 12, 13]
available_nodes_id [12, 13, 14]
available_nodes_id [12, 13, 14]
available_nodes_id [12, 13, 14]
available_nodes_id [13, 14, 15, 26]
available_nodes_id [14, 15, 27]
available_nodes_id [15, 26, 27, 29]
available_nodes_id [27, 29]
available_nodes_id [29]
いった
o_id, d_id, last_tt 8 39.0 9
Ti= 9 len(nonzero_xgrav) 9
available_nodes_id [8, 9]
available_nodes_id [8, 9]
available_nodes_id [8, 9, 10, 31]
available_nodes_id [9, 10, 11, 37]
available_nodes_id [10, 37]
available_nodes_id [37, 38]
available_nodes_id [38, 39]
available_no

上で出した簡単な経路データからdf_transitを作るパート

df_diaryも久しぶりに引っ張り出してくる必要があるな．．．

In [39]:
# さっき作ったデータを読み込む
df_trajectory = pd.read_csv(os.path.join(basepath, date_stamp, f'transit_{date_stamp}30_20sec_off.csv'))

# 遷移回数カウント用のデータフレームを初期化
# N = df_trajectory['node'].nunique()
# node_ids = df_trajectory['node'].unique()
nodeid_list_array = np.array(nodeid_list)
transition_counts = pd.DataFrame(0, index=nodeid_list_array, columns=nodeid_list_array, dtype=int)

# 各個人の遷移をカウント
for _, group in df_trajectory.groupby('id'):
    for i in range(len(group) - 1):
        current_node = group.iloc[i]['node']
        next_node = group.iloc[i + 1]['node']
        transition_counts.at[current_node, next_node] += 1

# 遷移確率の計算
transition_probabilities = transition_counts.div(transition_counts.sum(axis=1), axis=0).fillna(0)

np.set_printoptions(threshold=np.inf)
print(transition_probabilities)

     1         2         3         4         5         6         7    \
1    0.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2    0.0  0.000000  0.095238  0.000000  0.000000  0.000000  0.000000   
3    0.0  0.061489  0.236246  0.446602  0.000000  0.000000  0.000000   
4    0.0  0.000000  0.157143  0.161905  0.019048  0.000000  0.000000   
5    0.0  0.000000  0.000000  0.200000  0.200000  0.600000  0.000000   
6    0.0  0.000000  0.000000  0.000000  0.000000  0.555556  0.111111   
7    0.0  0.000000  0.000000  0.000000  0.000000  1.000000  0.000000   
8    0.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
9    0.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
10   0.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
11   0.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
12   0.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
13   0.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.00

In [40]:
transition_probabilities.to_csv(os.path.join(basepath, date_stamp, f'transit_probability_{date_stamp}30_20sec_off.csv'))

In [119]:
# 上に加えてweb diaryの結果から改札→目的地の経験分布を出す
# df_diray = pd.read_csv('/Users/takahiromatsunaga/res2023/PPcameraTG/PP/PP_webq/q_ans/diary_cleaned_odcar_odlatlon_point.csv')
# これでいいと思ってたけど情報量落とし過ぎてたので大元のデータ使う

# d_gateとdの組み合わせを同様にカウントしてmatrix作る
df = pd.read_csv('/Users/takahiromatsunaga/res2023/PPcameraTG/PP/PP_webq/q_ans/diary.csv', encoding='shift-jis')
df_diary2 = pd.read_csv('/Users/takahiromatsunaga/res2023/PPcameraTG/PP/PP_webq/q_ans/diary_cleaned_odcar_odlatlon.csv')
df_multi_node = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/ver0707/multi_node_post.csv')
df_multi_link = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/ver0707/multi_link_post.csv')

df_diary = df.copy()
df_diary['id'] = df_diary['id'].str.replace('jrs', '', regex=True) # .astype(int)
df_diary['id'] = df_diary['id'].apply(lambda x: f"23{x.zfill(3)}").astype(int)

# date列をdatetimeに書き換える
df_diary['day'] = df_diary['day'].str.extract(r'(\d{1,2}/\d{1,2})') # 正規表現で合致する部分を抽出
df_diary['day'] = pd.to_datetime(df_diary['day'], format='%m/%d').dt.strftime('%m/%d')
df_diary = df_diary.reset_index(drop=True)

multi_nodeid_list = sorted(df_multi_node['nodeid'].unique())
multi_nodeid_list_array = np.array(multi_nodeid_list)


d_dict = {'渋谷ヒカリエ':52,'渋谷109':54,'渋谷フクラス':55,'渋谷マークシティ':53,'渋谷ストリーム':56} # ,'渋谷駅前スクランブル交差点':59} 渋谷スクランブルスクエア':60,'
gate_dict = {'1（南改札 東口）':35, '2（南改札 西口）':36, '3（ハチ公改札 宮益坂口）':29, '4（ハチ公改札 スクランブル交差点方面）':29, '5（ハチ公改札 ハチ公口）':28, '6（中央改札 東口）':39, '7（中央改札 京王井の頭線方面）':39, '8（中央改札 銀座線乗換）':39} #, '9（新南口）':0, '上記以外の出入り口を利用':0}

In [44]:
print(len(df_diary))

1579


In [48]:
## df_diary2のcrsをepsg6677に変換してdf_nodeの形式に揃える
import geopandas as gpd
from shapely.geometry import Point

# GeoDataFrameに変換
geometry = [Point(xy) for xy in zip(df_diary2['d_lon'], df_diary2['d_lat'])]
gdf = gpd.GeoDataFrame(df_diary2, geometry=geometry)

# 現在のCRSを設定 (ここではデフォルトのEPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)

# CRSをEPSG:6677に変換
gdf = gdf.to_crs(epsg=6677)


In [49]:
df_diary2.head()

,Unnamed: 0,id,day,time,purpose,d,o,d_line,d_car,staire_to_gate,...,o_line,o_gate,staire_to_home,o_car,dcar_node,ocar_node,d_lat,d_lon,o_lat,o_lon
0,0,23026,12/19,10:00-11:00,2,7,5,1,4.0,12,...,5,0,NaN,NaN,12,0,35.659938,139.700059,35.659624,139.699793
1,1,23020,12/21,13:00-14:00,11,0,0,10,NaN,NaN,...,0,0,NaN,NaN,0,0,0.000000,0.000000,0.000000,0.000000
2,2,23034,12/20,14:00-15:00,0,7,7,1,3.0,12,...,2,1,5,2.0,13,23,35.661266,139.698110,35.661194,139.698300
3,3,23244,12/21,18:00-19:00,4,0,0,1,4.0,12,...,9,0,NaN,NaN,12,0,0.000000,0.000000,35.659481,139.701597
4,4,23120,12/19,6:00-7:00,2,0,0,1,1.0,12,...,0,0,NaN,NaN,15,0,0.000000,0.000000,0.000000,0.000000


In [77]:
# 結果の確認
gdf.head()


,Unnamed: 0,id,day,time,purpose,d,o,d_line,d_car,staire_to_gate,...,o_gate,staire_to_home,o_car,dcar_node,ocar_node,d_lat,d_lon,o_lat,o_lon,geometry
0,0,23026,12/19,10:00-11:00,2,7,5,1,4.0,12,...,0,NaN,NaN,12,0,35.659938,139.700059,35.659624,139.699793,POINT (-12066.679 -37719.915)
1,1,23020,12/21,13:00-14:00,11,0,0,10,NaN,NaN,...,0,NaN,NaN,0,0,0.000000,0.000000,0.000000,0.000000,POINT (-4893343.728 16016786.949)
2,2,23034,12/20,14:00-15:00,0,7,7,1,3.0,12,...,1,5,2.0,13,23,35.661266,139.698110,35.661194,139.698300,POINT (-12242.94 -37572.343)
3,3,23244,12/21,18:00-19:00,4,0,0,1,4.0,12,...,0,NaN,NaN,12,0,0.000000,0.000000,35.659481,139.701597,POINT (-4893343.728 16016786.949)
4,4,23120,12/19,6:00-7:00,2,0,0,1,1.0,12,...,0,NaN,NaN,15,0,0.000000,0.000000,0.000000,0.000000,POINT (-4893343.728 16016786.949)


In [55]:
print(gdf.loc[0, 'geometry'])
print(type(gdf.loc[0, 'geometry']))
print(np.array(gdf.loc[0, 'geometry']))
print(type(np.array(gdf.loc[0, 'geometry'])))

POINT (-12066.679360435159 -37719.91460144872)
<class 'shapely.geometry.point.Point'>
POINT (-12066.679360435159 -37719.91460144872)
<class 'numpy.ndarray'>


In [62]:
a = np.array(gdf.loc[0, 'geometry'])
aa = gdf.loc[0, 'geometry']
arrya = np.array([aa.x, aa.y])
# coordinates = np.array([[geom.x, geom.y] for geom in gdf.geometry])
# coordinates = np.array([[geom.x, geom.y] in gdf.loc[0, 'geometry']])

print(arrya)
print(type(arrya))

[-12066.67936044 -37719.91460145]
<class 'numpy.ndarray'>


In [46]:
list(d_dict.keys())

['渋谷ヒカリエ', '渋谷109', '渋谷フクラス', '渋谷マークシティ', '渋谷ストリーム']

In [86]:
# df_nodeから，外のnodeについてのみtreeを作る
from scipy.spatial import cKDTree

mezzo_nodeid_list = [i for i in range(51, 61)]
df_node_nonabs = df_multi_node.copy()
df_node_mezzo = df_node_nonabs[df_node_nonabs['nodeid'].isin(mezzo_nodeid_list)]
node_positions_mezzo = df_node_mezzo[['x', 'y']].values
kdtree_mezzo = cKDTree(node_positions_mezzo) # KDTreeを構築

In [87]:
print(df_node_mezzo)

    nodeid  floor          x          y
30      51   14.8 -11889.429 -37548.534
31      52   28.5 -11795.765 -37830.511
32      53   28.5 -12083.169 -37862.506
33      54   14.8 -12159.666 -37761.208
34      55   28.5 -12053.557 -37951.112
35      56   28.5 -11822.109 -37999.642
36      57   14.8 -11930.523 -38065.246
37      58   14.8 -11976.777 -37508.895
38      59   14.8 -12113.741 -37610.256
39      60   14.8 -11887.702 -37878.546


In [88]:
df_diary = df_diary.fillna(0)
print(df_diary.head(10))

      id    day         time    purpose              d             o  \
0  23026  12/19  10:00-11:00  仕事（勤務先以外）  渋谷駅前スクランブル交差点      渋谷マークシティ   
1  23020  12/21  13:00-14:00     乗り換えのみ              0             0   
2  23034  12/20  14:00-15:00         趣味  渋谷駅前スクランブル交差点   最初に訪れた場所と同じ   
3  23244  12/21  18:00-19:00         帰宅           特になし          特になし   
4  23120  12/19    6:00-7:00  仕事（勤務先以外）        通り過ぎただけ   そのまま会社まで歩いた   
5  23005  12/22  17:00-18:00  仕事（勤務先以外）   渋谷スクランブルスクエア   最初に訪れた場所と同じ   
6  23010  12/19  12:00-13:00    仕事（勤務先）       渋谷マークシティ   最初に訪れた場所と同じ   
7  23050  12/19  18:00-19:00         散策  渋谷駅前スクランブル交差点  渋谷スクランブルスクエア   
8  23231  12/20  13:00-14:00     乗り換えのみ              0             0   
9  23167  12/19  18:00-19:00         買物        ミヤシタパーク   最初に訪れた場所と同じ   

             d_line  d_car staire_to_gate                d_gate  \
0  山手線内回り（池袋→渋谷→大崎）    4.0             12  4（ハチ公改札 スクランブル交差点方面）   
1            京王井の頭線    0.0              0                     0   
2  山手線

In [123]:
import random

gate_node = 0
destination_node = 0

count = 0
multi_transition_counts = pd.DataFrame(0, index=multi_nodeid_list_array, columns=multi_nodeid_list_array, dtype=int)

# 各個人の遷移をカウント
for i in range(len(df_diary)):
    gate = df_diary.loc[i, 'd_gate']
    destination = df_diary.loc[i, 'd']
    day = df_diary.loc[i, 'day']
    id_value = df_diary.loc[i, 'id']
    time = df_diary.loc[i, 'time']

    # そもそも回答がnanだったらスキップ
    if gate == 0:
        continue
    if destination == 0:
        continue
    
    ##### 改札はシンプル #####
    gate_key_list = list(gate_dict.keys()) 

    if gate in gate_key_list:
        gate_node = gate_dict[gate]
        print('ok', gate)

    elif gate not in gate_key_list:
        print('だめ', gate)
        continue
    
    # if gate_node == 0:
    #     continue 
    
    ##### 目的地がめんどくさい #####
    destination_key_list = list(d_dict.keys()) 

    if destination in destination_key_list:
        destination_node = d_dict[destination]
    
    else: # 辞書にない場合
        if destination == '渋谷スクランブルスクエア':
            ran = random.random()
            if ran <= 0.5:
                destination_node = 60
            else:
                destination_node = 52

        elif destination == '渋谷駅前スクランブル交差点': # 配分
            ran = random.random()
            if ran <= 0.33:
                destination_node = 54
            elif 0.33 <= ran <= 0.66:
                destination_node = 59
            else:
                destination_node = 58
        
        else: # それ以外の目的地を持っている→df_diary2→gdfを参照
            # gdf から一致する行をフィルタリング
            matching_row = gdf[(gdf['day'] == day) & (gdf['id'] == id_value) & (gdf['time'] == time)]
            
            if not matching_row.empty:
                d_geo = matching_row['geometry'].values[0]
                d_loc = np.array((d_geo.x, d_geo.y)) # これでいいはず
                # d_locに最寄りのノードをkdtree_mezzoから抽出
                dist_mezzo, idx_mezzo = kdtree_mezzo.query(d_loc) 
                destination_node = mezzo_nodeid_list[idx_mezzo] # idを出す

                if d_loc[1] >= 0:
                    print(f'd_loc={d_loc}なのでだめ')
                    continue
                
            else:
                continue
    print(d_loc, destination_node)
    count += 1
    ## 以上でgate_nodeとdestination_nodeが出揃った
    ## カウントして遷移確率を出す
    multi_transition_counts.at[gate_node, destination_node] += 1
print(count)

ok 4（ハチ公改札 スクランブル交差点方面）
[-4893343.7277767  16016786.94928586] 54
だめ nan
ok 4（ハチ公改札 スクランブル交差点方面）
[-4893343.7277767  16016786.94928586] 54
ok 5（ハチ公改札 ハチ公口）
d_loc=[-4893343.7277767  16016786.94928586]なのでだめ
ok 3（ハチ公改札 宮益坂口）
d_loc=[-4893343.7277767  16016786.94928586]なのでだめ
だめ nan
だめ nan
ok 4（ハチ公改札 スクランブル交差点方面）
[-4893343.7277767  16016786.94928586] 54
ok 5（ハチ公改札 ハチ公口）
[-11982.85569988 -37799.57322046] 53
ok 5（ハチ公改札 ハチ公口）
[-11921.83363985 -37598.85032316] 51
ok 2（南改札 西口）
[-12021.40125643 -37915.12268139] 55
ok 6（中央改札 東口）
d_loc=[-4893343.7277767  16016786.94928586]なのでだめ
ok 1（南改札 東口）
[-11916.61437154 -37893.18618404] 60
だめ nan
だめ nan
だめ nan
ok 7（中央改札 京王井の頭線方面）
d_loc=[-4893343.7277767  16016786.94928586]なのでだめ
ok 3（ハチ公改札 宮益坂口）
[-4893343.7277767  16016786.94928586] 60
だめ nan
ok 2（南改札 西口）
d_loc=[-4893343.7277767  16016786.94928586]なのでだめ
ok 5（ハチ公改札 ハチ公口）
[-4893343.7277767  16016786.94928586] 52
ok 2（南改札 西口）
d_loc=[-4893343.7277767  16016786.94928586]なのでだめ
ok 1（南改札 東口）
d_loc=[-4893343.7277767  160167

In [125]:
z = mezzo_nodeid_list.index(51)
print(z)
print(multi_transition_counts.iloc[:-10, :])
#print(multi_transition_counts.iloc[:, 51:60])
multi_transition_counts.to_csv('/Users/takahiromatsunaga/res2023/PPcameraTG/PP/PP_webq/q_ans/multi_transit_count.csv')

0
    1    2    3    4    5    6    7    8    9    10   ...  52   53   54   55   \
1     0    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
2     0    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
3     0    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
4     0    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
5     0    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
6     0    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
7     0    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
8     0    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
9     0    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
10    0    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
11    0    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
12    0    0    0    0    

In [126]:

# 遷移確率の計算
multi_transition_probabilities = multi_transition_counts.div(multi_transition_counts.sum(axis=1), axis=0).fillna(0)

np.set_printoptions(threshold=np.inf)
print(multi_transition_probabilities)

     1    2    3    4    5    6    7    8    9    10   ...       52   \
1    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.000000   
2    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.000000   
3    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.000000   
4    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.000000   
5    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.000000   
6    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.000000   
7    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.000000   
8    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.000000   
9    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.000000   
10   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.000000   
11   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.000000   
12   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.000000   
13   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.00

In [127]:
multi_transition_probabilities.to_csv('/Users/takahiromatsunaga/res2023/PPcameraTG/PP/PP_webq/q_ans/multi_transit_probability.csv')

In [128]:
import pandas as pd

# 例としてnodeid_listを定義
nodeid_list = ['node1', 'node2', 'node3', 'node4', 'node5']

# 例としてのDataFrame b (M*M)
data_b = [[0, 1, 2, 3, 4],
          [1, 0, 1, 2, 3],
          [2, 1, 0, 1, 2],
          [3, 2, 1, 0, 1],
          [4, 3, 2, 1, 0]]

b = pd.DataFrame(data_b, index=nodeid_list, columns=nodeid_list)

# 例としてのDataFrame a (N*N)
data_a = [[0, 1, 2],
          [1, 0, 1],
          [2, 1, 0]]

# N個のノードのみ使用（ここでは最初の3つ）
nodeid_list_a = nodeid_list[:3]

a = pd.DataFrame(data_a, index=nodeid_list_a, columns=nodeid_list_a)

# bの一部をaで更新
b.update(a)

# 結果を表示
print(b)


       node1  node2  node3  node4  node5
node1    0.0    1.0    2.0      3      4
node2    1.0    0.0    1.0      2      3
node3    2.0    1.0    0.0      1      2
node4    3.0    2.0    1.0      0      1
node5    4.0    3.0    2.0      1      0


In [129]:
# 大きいdataframeの一部を小さいdataframeで置き換える
multi_transition_probabilities.update(transition_probabilities)


In [130]:
multi_transition_probabilities.to_csv(os.path.join(basepath, date_stamp, f'fusioned_transit_probability_{date_stamp}30_20sec_off.csv'))
# transition_probabilities.to_csv(os.path.join(basepath, date_stamp, f'transit_probability_{date_stamp}30_20sec_off.csv'))

In [6]:
import numpy as np 
c = np.array([3, 3, 4, 5, 5, 1, 11])
c[:2]

array([3, 3])

In [5]:
c[1:]

array([ 3,  4,  5,  5,  1, 11])

In [7]:
d = np.array((9, 2, 4))

In [10]:
result = np.concatenate((c, d))
print(result)

[ 3  3  4  5  5  1 11  9  2  4]


In [11]:
print(type(result))

<class 'numpy.ndarray'>
